In [2]:
"""
COMPLETE FIXED Multi-Task RoBERTa Model for Sarcasm Detection and Emotion Classification
Optimized for better emotion learning with simplified architecture
"""

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             confusion_matrix, classification_report, roc_auc_score)
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("FIXED MULTI-TASK ROBERTA: SARCASM + EMOTION CLASSIFICATION")
print("="*80)

# Set random seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ============================================================================
# 1. LOAD AND PREPARE DATA WITH EMOTION GROUPING
# ============================================================================
print("\n" + "="*80)
print("1. LOADING AND PREPARING DATA")
print("="*80)

# Load your dataset
df = pd.read_csv('/content/final_clean_no_duplicates.csv')  # MODIFY THIS PATH
print(f"Loaded dataset: {df.shape[0]} rows")

# Text preprocessing
print("\nPreprocessing text...")
df['headline_clean'] = df['headline'].str.strip()

# Define emotion grouping (27 → 8 classes)
emotion_mapping = {
    # Positive emotions
    'admiration': 'positive',
    'excitement': 'positive',
    'hope': 'positive',
    'joy': 'positive',
    'love': 'positive',
    'empowerment': 'positive',
    'relaxation': 'positive',

    # Negative emotions
    'anger': 'negative',
    'contempt': 'negative',
    'disapproval': 'negative',
    'embarrassment': 'negative',
    'fear': 'negative',
    'sadness': 'negative',
    'concern': 'negative',

    # Complex/Neutral emotions
    'activism': 'complex',
    'anticipation': 'complex',
    'curiosity': 'complex',
    'drama': 'complex',
    'intensity': 'complex',
    'reflection': 'complex',
    'solidarity': 'complex',
    'sincerity': 'complex',

    # Specific categories
    'humor': 'humor',
    'somber irony': 'irony',
    'surprise': 'surprise',
    'neutral': 'neutral',
    'nostalgia': 'nostalgia'
}

# Apply emotion grouping
df['emotion_grouped'] = df['emotion'].map(emotion_mapping)

# Check for any unmapped emotions
unmapped = df[df['emotion_grouped'].isnull()]['emotion'].unique()
if len(unmapped) > 0:
    print(f"Warning: Unmapped emotions: {unmapped}")
    # Fill unmapped with 'complex'
    df['emotion_grouped'] = df['emotion_grouped'].fillna('complex')

# Encode grouped emotion labels
label_encoder = LabelEncoder()
df['emotion_encoded'] = label_encoder.fit_transform(df['emotion_grouped'])
emotion_classes = label_encoder.classes_
num_emotions = len(emotion_classes)

print(f"\nTarget Variables:")
print(f"  - is_sarcastic: Binary (0/1)")
print(f"  - emotion: {num_emotions} classes (reduced from 27)")
print(f"  Emotion classes: {list(emotion_classes)}")

# Display emotion distribution
print(f"\nEmotion Distribution:")
emotion_counts = df['emotion_grouped'].value_counts()
for emotion, count in emotion_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {emotion:15s}: {count:4d} samples ({percentage:.1f}%)")

# ============================================================================
# 2. DATA SPLITTING
# ============================================================================
print("\n" + "="*80)
print("2. CREATING DATA SPLITS")
print("="*80)

X = df['headline_clean'].values
y_sarcasm = df['is_sarcastic'].values
y_emotion = df['emotion_encoded'].values

# Create combined labels for better stratification
combined_labels = [f"{sarc}_{emo}" for sarc, emo in zip(y_sarcasm, y_emotion)]

# Split data
X_train, X_temp, y_sarcasm_train, y_sarcasm_temp, y_emotion_train, y_emotion_temp = train_test_split(
    X, y_sarcasm, y_emotion,
    test_size=0.2,
    random_state=SEED,
    stratify=combined_labels
)

X_val, X_test, y_sarcasm_val, y_sarcasm_test, y_emotion_val, y_emotion_test = train_test_split(
    X_temp, y_sarcasm_temp, y_emotion_temp,
    test_size=0.5,
    random_state=SEED,
    stratify=[f"{s}_{e}" for s, e in zip(y_sarcasm_temp, y_emotion_temp)]
)

print(f"Dataset Split:")
print(f"  Train set: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Val set:   {len(X_val)} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"  Test set:  {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")

# ============================================================================
# 3. TOKENIZATION
# ============================================================================
print("\n" + "="*80)
print("3. TOKENIZING WITH ROBERTA")
print("="*80)

print("Loading RoBERTa tokenizer...")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

MAX_LENGTH = 128
print(f"Max sequence length: {MAX_LENGTH}")

class HeadlineDataset(Dataset):
    def __init__(self, texts, sarcasm_labels, emotion_labels, tokenizer, max_length):
        self.texts = texts
        self.sarcasm_labels = sarcasm_labels
        self.emotion_labels = emotion_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        sarcasm = self.sarcasm_labels[idx]
        emotion = self.emotion_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'sarcasm_label': torch.tensor(sarcasm, dtype=torch.long),
            'emotion_label': torch.tensor(emotion, dtype=torch.long)
        }

print("Creating datasets...")
train_dataset = HeadlineDataset(X_train, y_sarcasm_train, y_emotion_train, tokenizer, MAX_LENGTH)
val_dataset = HeadlineDataset(X_val, y_sarcasm_val, y_emotion_val, tokenizer, MAX_LENGTH)
test_dataset = HeadlineDataset(X_test, y_sarcasm_test, y_emotion_test, tokenizer, MAX_LENGTH)

BATCH_SIZE = 32  # Increased batch size for stability
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Batch size: {BATCH_SIZE}")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

# ============================================================================
# 4. SIMPLIFIED MODEL ARCHITECTURE
# ============================================================================
print("\n" + "="*80)
print("4. BUILDING SIMPLIFIED MULTI-TASK MODEL")
print("="*80)

class SimpleMultiTaskRoBERTa(nn.Module):
    def __init__(self, num_emotions, dropout=0.1):
        super(SimpleMultiTaskRoBERTa, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(dropout)

        # Simple direct classification heads
        self.sarcasm_classifier = nn.Linear(768, 2)
        self.emotion_classifier = nn.Linear(768, num_emotions)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        sarcasm_logits = self.sarcasm_classifier(pooled_output)
        emotion_logits = self.emotion_classifier(pooled_output)

        return sarcasm_logits, emotion_logits

print("Loading simplified RoBERTa model...")
model = SimpleMultiTaskRoBERTa(num_emotions=num_emotions).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# ============================================================================
# 5. OPTIMIZED TRAINING SETUP
# ============================================================================
print("\n" + "="*80)
print("5. TRAINING CONFIGURATION")
print("="*80)

EPOCHS = 6
INITIAL_EMOTION_WEIGHT = 0.8  # Start with focus on emotion
INITIAL_SARCASM_WEIGHT = 0.2

# Simple loss functions without class weights (for stability)
criterion_sarcasm = nn.CrossEntropyLoss()
criterion_emotion = nn.CrossEntropyLoss()

# Optimizer with different learning rates
optimizer = AdamW([
    {'params': model.roberta.parameters(), 'lr': 2e-5},
    {'params': model.sarcasm_classifier.parameters(), 'lr': 1e-4},
    {'params': model.emotion_classifier.parameters(), 'lr': 5e-4}  # Higher LR for emotion
], weight_decay=0.01)

# Learning rate scheduler
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

print(f"Epochs: {EPOCHS}")
print(f"Initial task weights - Emotion: {INITIAL_EMOTION_WEIGHT}, Sarcasm: {INITIAL_SARCASM_WEIGHT}")
print(f"Learning rates - RoBERTa: 2e-5, Sarcasm: 1e-4, Emotion: 5e-4")
print(f"Total training steps: {total_steps}")

# ============================================================================
# 6. IMPROVED TRAINING LOOP
# ============================================================================
print("\n" + "="*80)
print("6. STARTING TRAINING")
print("="*80)

def train_epoch(model, data_loader, optimizer, scheduler, device, epoch):
    model.train()
    total_loss = 0
    sarcasm_correct = 0
    emotion_correct = 0
    total_samples = 0

    # Dynamic weighting: Gradually balance the tasks
    emotion_weight = max(INITIAL_EMOTION_WEIGHT - (epoch * 0.1), 0.4)
    sarcasm_weight = 1.0 - emotion_weight

    for batch_idx, batch in enumerate(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        sarcasm_labels = batch['sarcasm_label'].to(device)
        emotion_labels = batch['emotion_label'].to(device)

        optimizer.zero_grad()

        sarcasm_logits, emotion_logits = model(input_ids, attention_mask)

        loss_sarcasm = criterion_sarcasm(sarcasm_logits, sarcasm_labels)
        loss_emotion = criterion_emotion(emotion_logits, emotion_labels)

        # Dynamic weighted loss
        loss = sarcasm_weight * loss_sarcasm + emotion_weight * loss_emotion

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        # Calculate accuracy
        sarcasm_preds = torch.argmax(sarcasm_logits, dim=1)
        emotion_preds = torch.argmax(emotion_logits, dim=1)

        sarcasm_correct += (sarcasm_preds == sarcasm_labels).sum().item()
        emotion_correct += (emotion_preds == emotion_labels).sum().item()
        total_samples += sarcasm_labels.size(0)

        if (batch_idx + 1) % 50 == 0:
            current_sarcasm_acc = sarcasm_correct / total_samples
            current_emotion_acc = emotion_correct / total_samples
            print(f"  Batch {batch_idx+1:3d}/{len(data_loader)} | "
                  f"Loss: {loss.item():.4f} | "
                  f"Sarcasm: {current_sarcasm_acc:.4f} | "
                  f"Emotion: {current_emotion_acc:.4f} | "
                  f"Weights: S({sarcasm_weight:.1f}) E({emotion_weight:.1f})")

    avg_loss = total_loss / len(data_loader)
    sarcasm_acc = sarcasm_correct / total_samples
    emotion_acc = emotion_correct / total_samples

    return avg_loss, sarcasm_acc, emotion_acc, emotion_weight

def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0

    all_sarcasm_preds = []
    all_sarcasm_labels = []
    all_sarcasm_probs = []
    all_emotion_preds = []
    all_emotion_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            sarcasm_labels = batch['sarcasm_label'].to(device)
            emotion_labels = batch['emotion_label'].to(device)

            sarcasm_logits, emotion_logits = model(input_ids, attention_mask)

            loss_sarcasm = criterion_sarcasm(sarcasm_logits, sarcasm_labels)
            loss_emotion = criterion_emotion(emotion_logits, emotion_labels)
            loss = 0.5 * loss_sarcasm + 0.5 * loss_emotion  # Equal weights for evaluation

            total_loss += loss.item()

            sarcasm_probs = torch.softmax(sarcasm_logits, dim=1)
            sarcasm_preds = torch.argmax(sarcasm_logits, dim=1)
            emotion_preds = torch.argmax(emotion_logits, dim=1)

            all_sarcasm_preds.extend(sarcasm_preds.cpu().numpy())
            all_sarcasm_labels.extend(sarcasm_labels.cpu().numpy())
            all_sarcasm_probs.extend(sarcasm_probs[:, 1].cpu().numpy())
            all_emotion_preds.extend(emotion_preds.cpu().numpy())
            all_emotion_labels.extend(emotion_labels.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    return (avg_loss,
            np.array(all_sarcasm_preds), np.array(all_sarcasm_labels), np.array(all_sarcasm_probs),
            np.array(all_emotion_preds), np.array(all_emotion_labels))

# Training with early stopping
best_val_emotion_acc = 0
patience = 3
patience_counter = 0
best_model_state = None

training_history = []

print("\nStarting training...")
for epoch in range(EPOCHS):
    print(f"\n{'='*80}")
    print(f"EPOCH {epoch + 1}/{EPOCHS}")
    print(f"{'='*80}")

    # Training
    train_loss, train_sarcasm_acc, train_emotion_acc, current_emotion_weight = train_epoch(
        model, train_loader, optimizer, scheduler, device, epoch
    )

    # Validation
    val_loss, val_sarcasm_preds, val_sarcasm_labels, val_sarcasm_probs, val_emotion_preds, val_emotion_labels = evaluate(
        model, val_loader, device
    )

    val_sarcasm_acc = accuracy_score(val_sarcasm_labels, val_sarcasm_preds)
    val_emotion_acc = accuracy_score(val_emotion_labels, val_emotion_preds)

    print(f"\nEpoch {epoch + 1} Summary:")
    print(f"  Training   -> Loss: {train_loss:.4f}, Sarcasm: {train_sarcasm_acc:.4f}, Emotion: {train_emotion_acc:.4f}")
    print(f"  Validation -> Loss: {val_loss:.4f}, Sarcasm: {val_sarcasm_acc:.4f}, Emotion: {val_emotion_acc:.4f}")
    print(f"  Emotion weight: {current_emotion_weight:.2f}")

    # Save training history
    training_history.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_sarcasm_acc': train_sarcasm_acc,
        'train_emotion_acc': train_emotion_acc,
        'val_loss': val_loss,
        'val_sarcasm_acc': val_sarcasm_acc,
        'val_emotion_acc': val_emotion_acc
    })

    # Early stopping based on emotion accuracy
    if val_emotion_acc > best_val_emotion_acc:
        best_val_emotion_acc = val_emotion_acc
        patience_counter = 0
        best_model_state = model.state_dict().copy()
        print(f"  ✓ New best model! Emotion accuracy: {val_emotion_acc:.4f}")

        # Save model checkpoint
        torch.save(model.state_dict(), f'best_model_epoch_{epoch+1}.pt')
    else:
        patience_counter += 1
        print(f"  × No improvement ({patience_counter}/{patience})")

    if patience_counter >= patience and epoch >= 2:  # Minimum 3 epochs
        print(f"\nEarly stopping triggered after {epoch + 1} epochs!")
        model.load_state_dict(best_model_state)
        break

# Load best model for final evaluation
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("Loaded best model for final evaluation.")

# ============================================================================
# 7. FINAL EVALUATION ON TEST SET
# ============================================================================
print("\n" + "="*80)
print("7. FINAL TEST SET EVALUATION")
print("="*80)

test_loss, sarcasm_preds, sarcasm_labels, sarcasm_probs, emotion_preds, emotion_labels = evaluate(
    model, test_loader, device
)

print(f"\nTest Loss: {test_loss:.4f}")

# ============================================================================
# 8. SARCASM DETECTION METRICS
# ============================================================================
print("\n" + "="*80)
print("8. SARCASM DETECTION RESULTS")
print("="*80)

sarcasm_acc = accuracy_score(sarcasm_labels, sarcasm_preds)
sarcasm_precision, sarcasm_recall, sarcasm_f1, _ = precision_recall_fscore_support(
    sarcasm_labels, sarcasm_preds, average='binary'
)
sarcasm_auc = roc_auc_score(sarcasm_labels, sarcasm_probs)

print(f"\nSarcasm Detection Metrics:")
print(f"  Accuracy:  {sarcasm_acc:.4f}")
print(f"  Precision: {sarcasm_precision:.4f}")
print(f"  Recall:    {sarcasm_recall:.4f}")
print(f"  F1-Score:  {sarcasm_f1:.4f}")
print(f"  ROC-AUC:   {sarcasm_auc:.4f}")

print(f"\nConfusion Matrix:")
cm_sarcasm = confusion_matrix(sarcasm_labels, sarcasm_preds)
print(f"                Predicted")
print(f"               Non-Sarc  Sarcastic")
print(f"Actual Non-Sarc    {cm_sarcasm[0][0]:4d}      {cm_sarcasm[0][1]:4d}")
print(f"Actual Sarcastic   {cm_sarcasm[1][0]:4d}      {cm_sarcasm[1][1]:4d}")

# ============================================================================
# 9. EMOTION CLASSIFICATION METRICS
# ============================================================================
print("\n" + "="*80)
print("9. EMOTION CLASSIFICATION RESULTS")
print("="*80)

emotion_acc = accuracy_score(emotion_labels, emotion_preds)
emotion_precision, emotion_recall, emotion_f1, _ = precision_recall_fscore_support(
    emotion_labels, emotion_preds, average='macro'
)
emotion_precision_w, emotion_recall_w, emotion_f1_w, _ = precision_recall_fscore_support(
    emotion_labels, emotion_preds, average='weighted'
)

print(f"\nEmotion Classification Metrics:")
print(f"  Accuracy:           {emotion_acc:.4f}")
print(f"  Macro Precision:    {emotion_precision:.4f}")
print(f"  Macro Recall:       {emotion_recall:.4f}")
print(f"  Macro F1-Score:     {emotion_f1:.4f}")
print(f"  Weighted F1-Score:  {emotion_f1_w:.4f}")

print(f"\nPer-Emotion Performance:")
emotion_report = classification_report(
    emotion_labels, emotion_preds,
    target_names=emotion_classes,
    digits=4
)
print(emotion_report)

# ============================================================================
# 10. CONFUSION ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("10. CONFUSION ANALYSIS")
print("="*80)

cm_emotion = confusion_matrix(emotion_labels, emotion_preds)

# Find top confusions
confusions = []
for i in range(len(emotion_classes)):
    for j in range(len(emotion_classes)):
        if i != j and cm_emotion[i][j] > 0:
            confusions.append((emotion_classes[i], emotion_classes[j], cm_emotion[i][j]))

confusions.sort(key=lambda x: x[2], reverse=True)

print("\nMost Common Misclassifications:")
for idx, (true_label, pred_label, count) in enumerate(confusions[:10], 1):
    print(f"  {idx:2d}. {true_label:15s} → {pred_label:15s}: {count:3d} times")

# ============================================================================
# 11. SAMPLE PREDICTIONS
# ============================================================================
print("\n" + "="*80)
print("11. SAMPLE PREDICTIONS")
print("="*80)

sample_indices = np.random.choice(len(X_test), min(8, len(X_test)), replace=False)

print("\nRandom Test Samples:")
for idx in sample_indices:
    true_sarcasm = sarcasm_labels[idx]
    pred_sarcasm = sarcasm_preds[idx]
    true_emotion = emotion_classes[emotion_labels[idx]]
    pred_emotion = emotion_classes[emotion_preds[idx]]
    headline = X_test[idx]

    sarcasm_match = "✓" if true_sarcasm == pred_sarcasm else "✗"
    emotion_match = "✓" if true_emotion == pred_emotion else "✗"

    print(f"\n  Headline: {headline}")
    print(f"  Sarcasm - True: {true_sarcasm}, Pred: {pred_sarcasm} {sarcasm_match}")
    print(f"  Emotion - True: {true_emotion}, Pred: {pred_emotion} {emotion_match}")

# ============================================================================
# 12. TRAINING SUMMARY
# ============================================================================
print("\n" + "="*80)
print("12. TRAINING SUMMARY")
print("="*80)

print(f"\nModel: Simplified Multi-Task RoBERTa")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")
print(f"Epochs completed: {len(training_history)}")

print(f"\nFinal Test Results:")
print(f"  Sarcasm Detection:")
print(f"    - Accuracy: {sarcasm_acc:.4f}")
print(f"    - F1-Score: {sarcasm_f1:.4f}")
print(f"    - ROC-AUC:  {sarcasm_auc:.4f}")
print(f"  Emotion Classification:")
print(f"    - Accuracy:      {emotion_acc:.4f}")
print(f"    - Macro F1:      {emotion_f1:.4f}")
print(f"    - Weighted F1:   {emotion_f1_w:.4f}")

# ============================================================================
# 13. SAVE MODEL FOR WEB APP
# ============================================================================
print("\n" + "="*80)
print("13. SAVING MODEL FOR DEPLOYMENT")
print("="*80)

save_dir = "fixed_multitask_roberta"

import os
os.makedirs(save_dir, exist_ok=True)

# Save tokenizer
print("Saving tokenizer...")
tokenizer.save_pretrained(save_dir)

# Save model weights
print("Saving model weights...")
torch.save(model.state_dict(), os.path.join(save_dir, "pytorch_model.bin"))

# Save config
print("Saving model config...")
config = {
    "model_type": "roberta",
    "num_emotions": num_emotions,
    "hidden_size": 768,
    "emotion_classes": emotion_classes.tolist(),
    "emotion_mapping": emotion_mapping
}
import json
with open(os.path.join(save_dir, "config.json"), "w") as f:
    json.dump(config, f, indent=4)

# Save label encoder
import joblib
joblib.dump(label_encoder, os.path.join(save_dir, "label_encoder.pkl"))

print(f"\nModel successfully saved to: {save_dir}/")
print("Ready for web app deployment!")

print("\n" + "="*80)
print("TRAINING COMPLETED SUCCESSFULLY!")
print("="*80)

FIXED MULTI-TASK ROBERTA: SARCASM + EMOTION CLASSIFICATION

Using device: cuda
GPU: Tesla T4

1. LOADING AND PREPARING DATA
Loaded dataset: 13478 rows

Preprocessing text...

Target Variables:
  - is_sarcastic: Binary (0/1)
  - emotion: 8 classes (reduced from 27)
  Emotion classes: ['complex', 'humor', 'irony', 'negative', 'neutral', 'nostalgia', 'positive', 'surprise']

Emotion Distribution:
  complex        : 3952 samples (29.3%)
  negative       : 3549 samples (26.3%)
  positive       : 3407 samples (25.3%)
  nostalgia      :  543 samples (4.0%)
  irony          :  520 samples (3.9%)
  surprise       :  518 samples (3.8%)
  neutral        :  502 samples (3.7%)
  humor          :  487 samples (3.6%)

2. CREATING DATA SPLITS
Dataset Split:
  Train set: 10782 samples (80.0%)
  Val set:   1348 samples (10.0%)
  Test set:  1348 samples (10.0%)

3. TOKENIZING WITH ROBERTA
Loading RoBERTa tokenizer...
Max sequence length: 128
Creating datasets...
Batch size: 32
Training batches: 337
Valid

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Total parameters: 124,653,322
Trainable parameters: 124,653,322

5. TRAINING CONFIGURATION
Epochs: 6
Initial task weights - Emotion: 0.8, Sarcasm: 0.2
Learning rates - RoBERTa: 2e-5, Sarcasm: 1e-4, Emotion: 5e-4
Total training steps: 2022

6. STARTING TRAINING

Starting training...

EPOCH 1/6
  Batch  50/337 | Loss: 1.4941 | Sarcasm: 0.4581 | Emotion: 0.2075 | Weights: S(0.2) E(0.8)
  Batch 100/337 | Loss: 1.3757 | Sarcasm: 0.5141 | Emotion: 0.2791 | Weights: S(0.2) E(0.8)
  Batch 150/337 | Loss: 1.3635 | Sarcasm: 0.5831 | Emotion: 0.3442 | Weights: S(0.2) E(0.8)
  Batch 200/337 | Loss: 0.9668 | Sarcasm: 0.6345 | Emotion: 0.4003 | Weights: S(0.2) E(0.8)
  Batch 250/337 | Loss: 0.8318 | Sarcasm: 0.6704 | Emotion: 0.4494 | Weights: S(0.2) E(0.8)
  Batch 300/337 | Loss: 0.9055 | Sarcasm: 0.6999 | Emotion: 0.4867 | Weights: S(0.2) E(0.8)

Epoch 1 Summary:
  Training   -> Loss: 1.1673, Sarcasm: 0.7143, Emotion: 0.5091
  Validation -> Loss: 0.5481, Sarcasm: 0.8620, Emotion: 0.7300
  Emotion

In [3]:
# ============================================================================
# DOWNLOAD MODEL FILES FOR STREAMLIT
# ============================================================================
print("\n" + "="*80)
print("PREPARING FILES FOR STREAMLIT DEPLOYMENT")
print("="*80)

import shutil
import joblib

# Create a zip file with all model files
streamlit_files_dir = "streamlit_model_files"
os.makedirs(streamlit_files_dir, exist_ok=True)

# Copy all necessary files
print("Copying model files...")
shutil.copytree("fixed_multitask_roberta", os.path.join(streamlit_files_dir, "fixed_multitask_roberta"), dirs_exist_ok=True)

# Save the label encoder separately for easy loading
joblib.dump(label_encoder, os.path.join(streamlit_files_dir, "label_encoder.pkl"))

# Create a requirements.txt for Streamlit
requirements = """
torch>=1.9.0
transformers>=4.20.0
streamlit>=1.22.0
pandas>=1.3.0
numpy>=1.21.0
scikit-learn>=1.0.0
joblib>=1.1.0
""".strip()

with open(os.path.join(streamlit_files_dir, "requirements.txt"), "w") as f:
    f.write(requirements)

# Create a quick test script
test_script = """
# Quick test script for your model
import torch
import joblib
from transformers import RobertaTokenizer
from your_model_definition import SimpleMultiTaskRoBERTa

def test_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained('fixed_multitask_roberta')
    label_encoder = joblib.load('label_encoder.pkl')

    # Initialize model architecture
    model = SimpleMultiTaskRoBERTa(num_emotions=len(label_encoder.classes_))
    model.load_state_dict(torch.load('fixed_multitask_roberta/pytorch_model.bin', map_location=device))
    model.to(device)
    model.eval()

    # Test prediction
    test_text = "This is absolutely fantastic news!"
    inputs = tokenizer(test_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    with torch.no_grad():
        sarcasm_logits, emotion_logits = model(inputs['input_ids'], inputs['attention_mask'])

    sarcasm_pred = torch.argmax(sarcasm_logits, dim=1).item()
    emotion_pred = label_encoder.inverse_transform([torch.argmax(emotion_logits, dim=1).item()])[0]

    print(f"Text: {test_text}")
    print(f"Sarcastic: {bool(sarcasm_pred)}")
    print(f"Emotion: {emotion_pred}")

if __name__ == "__main__":
    test_model()
"""

with open(os.path.join(streamlit_files_dir, "test_model.py"), "w") as f:
    f.write(test_script)

# Create zip file
print("Creating zip file...")
shutil.make_archive("streamlit_model_files", 'zip', streamlit_files_dir)

print(f"\n✅ All files ready! Download 'streamlit_model_files.zip' from the Colab file browser.")
print("📁 Files included:")
print("   - fixed_multitask_roberta/ (model weights, tokenizer, config)")
print("   - label_encoder.pkl")
print("   - requirements.txt")
print("   - test_model.py")


PREPARING FILES FOR STREAMLIT DEPLOYMENT
Copying model files...
Creating zip file...

✅ All files ready! Download 'streamlit_model_files.zip' from the Colab file browser.
📁 Files included:
   - fixed_multitask_roberta/ (model weights, tokenizer, config)
   - label_encoder.pkl
   - requirements.txt
   - test_model.py
